## S&P Global - CORE2LOG: AUTOMATIC ROCK CLASSIFICATION
### Misael M. Morales, Carlos Torres-Verdin, and Michael Pyrcz, UT-Austin; Murray Christie, Vladimir Rabinovich, S&P Global
#### 2024, Digital Reservoir Characterization Technology (DiReCT)
***

In [ ]:
from auto_rock_classification import *
arc = RockClassification(well_number=623, method='gmm')
arc.run_dashboard()

***
# END

In [4]:
from auto_rock_classification import *

postprocess_dfs = []

arc = RockClassification(method='gmm', n_classes=3)
arc.bigloader()

for i in range(1379):
    arc.well_number = i
    arc.preprocessing(header=False)
    arc.calculate_method_clf()
    df = arc.d.copy()
    df['UWI'] = arc.uwi_clean[i]
    df['INTERVAL_DEPTH'] = arc.d.index
    df.to_csv('Data/UT Export core classification/{}.csv'.format(arc.uwi_clean[i]), index=False)
    postprocess_dfs.append(df)

postprocess_df = pd.concat(postprocess_dfs, ignore_index=True)
postprocess_df.to_csv('Data/Core2Log_RockClassification_all.csv', index=False)

All data shape: (102797, 88)
Total number of wells: 1379


In [ ]:
postprocess_df